In [2]:
import sys
!{sys.executable} -m pip install odfpy

  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=137344 sha256=27d9de28aa4854e5e53ed85cf08ed98b68348c3482495651b579514c3eaa5aec
  Stored in directory: c:\users\jpabl\appdata\local\pip\cache\wheels\ea\af\da\2bdd7308f7b334429a558df1e36d31864cd19c07ede92ddf0e
Successfully built odfpy


In [19]:
import pandas as pd
import numpy as np
from re import sub
import datetime as dt
import time
from decimal import Decimal

pd.options.display.max_columns = None

In [3]:
bookings = pd.read_excel("data/HWY40/Fareharbor/Bookings--2020-10-01--2021-03-13.ods", engine="odf", header=1)
bookings.head()

,Booking ID,Cancelled?,Created At,Created At Time,Created At Date,Created By,Last Booked At,Last Booked At Date,Last Booked At Time,Last Booked By,...,Availability ID,Availability Day,Headline,Private Headline,Contact Language,Subscribed to text messages?,Subscribed to Email?,# of Pax,Online Booking Reference,Pickup Time
0,#67251622,No,2020-12-17 @ 08:18pm,08:18pm,2020-12-17,Online,2020-12-18 @ 08:42am,2020-12-18,08:42am,Jami Turman,...,#531950708,Friday,Please select your first day of skiing! Book now!,NaN,English (US),No,No,2,NaN,NaN
1,#67152670,No,2020-12-16 @ 08:50am,08:50am,2020-12-16,Online,2020-12-16 @ 08:50am,2020-12-16,08:50am,Online,...,#531950708,Friday,Please select your first day of skiing! Book now!,NaN,English (US),Subscribed,No,2,NaN,NaN
2,#67251680,No,2020-12-17 @ 08:18pm,08:18pm,2020-12-17,Online,2020-12-18 @ 08:43am,2020-12-18,08:43am,Jami Turman,...,#532305509,Friday,Please select your first day of skiing! Book now!,NaN,English (US),No,No,2,NaN,NaN
3,#67152672,No,2020-12-16 @ 08:50am,08:50am,2020-12-16,Online,2020-12-16 @ 08:50am,2020-12-16,08:50am,Online,...,#532310577,Friday,Please select your first day of skiing! Book now!,NaN,English (US),Subscribed,No,5,NaN,NaN
4,#67354515,No,2020-12-16 @ 08:50am,08:50am,2020-12-16,Online,2020-12-19 @ 05:45pm,2020-12-19,05:45pm,Jami Turman,...,#532305510,Saturday,Please select your first day of skiing! Book now!,NaN,English (US),Subscribed,No,3,NaN,NaN


# Questions for bookings
- 2 columns completely missing?

In [4]:
campaigns = pd.read_excel("data/HWY40/Fareharbor/hwy40skidelivery-campaigns-summary-from-2020-10-11-to-2021-03-13-CLEAN.ods", engine="odf", sheet_name="Campaign_dates")
campaigns.head()

,campaign,Start,End
0,Save 50% off purchase,12/26/2020 @ 12:00am,1/1/2021 @ 12:00am
1,Skicoupons.com,1/21/21 @ 11:11am,NaN


In [5]:
campaign_totals = pd.read_excel("data/HWY40/Fareharbor/hwy40skidelivery-campaigns-summary-from-2020-10-11-to-2021-03-13-CLEAN.ods", engine="odf", sheet_name="Totals")
campaign_totals.head()

,Item,#,Subtotal,Tax,Total,% of Report Sum Booking Totals,Revenue Collected,Receivable from Affiliate,Payable to Affiliate,Net Revenue
0,Elite Ski Package,22,"$8,034.17",$456.05,"$8,490.22",15.7%,"$8,285.22",$0.00,$0.00,"$8,121.91"
1,Summit Ski Package,37,"$9,282.34",$517.45,"$9,799.79",18.13%,"$9,759.79",$0.00,$0.00,"$9,564.46"
2,Mountain Ski Package,100,"$20,099.29","$1,133.72","$21,233.01",39.27%,"$21,189.01",$0.00,$0.00,"$20,755.56"
3,Base Ski Package,36,"$5,988.14",$338.53,"$6,326.67",11.7%,"$6,307.67",$0.00,$0.00,"$6,177.18"
4,Junior Mountain Ski Package,14,"$2,528.45",$141.14,"$2,669.59",4.94%,"$2,669.59",$0.00,$0.00,"$2,616.14"


In [6]:
discount = pd.read_excel("data/HWY40/Fareharbor/hwy40skidelivery-campaigns-summary-from-2020-10-11-to-2021-03-13-CLEAN.ods", engine="odf", sheet_name="50%")

for i in range(len(discount)):
    discount['Subtotal'].iloc[i] = Decimal(sub(r'[^\d.]', '', discount['Subtotal'].iloc[i]))
    discount['Tax'].iloc[i] = Decimal(sub(r'[^\d.]', '', discount['Tax'].iloc[i]))
    discount['Total'].iloc[i] = Decimal(sub(r'[^\d.]', '', discount['Total'].iloc[i]))
    discount['Revenue Collected'].iloc[i] = Decimal(sub(r'[^\d.]', '', discount['Revenue Collected'].iloc[i]))
    discount['Receivable from Affiliate'].iloc[i] = Decimal(sub(r'[^\d.]', '', discount['Receivable from Affiliate'].iloc[i]))
    discount['Payable to Affiliate'].iloc[i] = Decimal(sub(r'[^\d.]', '', discount['Payable to Affiliate'].iloc[i]))
    discount['Net Revenue'].iloc[i] = Decimal(sub(r'[^\d.]', '', discount['Net Revenue'].iloc[i]))
    discount['% of Report Sum Booking Totals'].iloc[i] = float(sub('%', '', discount['% of Report Sum Booking Totals'].iloc[i]))

discount.head()

C:\Users\jpabl\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Item,#,Subtotal,Tax,Total,% of Report Sum Booking Totals,Revenue Collected,Receivable from Affiliate,Payable to Affiliate,Net Revenue
0,Elite Ski Package,21,7808.17,442.26,8250.43,15.26,8045.43,0.00,0.00,7886.98
1,Summit Ski Package,37,9282.34,517.45,9799.79,18.13,9759.79,0.00,0.00,9564.46
2,Mountain Ski Package,94,18891.74,1065.74,19957.48,36.92,19913.48,0.00,0.00,19506.07
3,Base Ski Package,36,5988.14,338.53,6326.67,11.7,6307.67,0.00,0.00,6177.18
4,Junior Mountain Ski Package,14,2528.45,141.14,2669.59,4.94,2669.59,0.00,0.00,2616.14


In [7]:
cupon = pd.read_excel("data/HWY40/Fareharbor/hwy40skidelivery-campaigns-summary-from-2020-10-11-to-2021-03-13-CLEAN.ods", engine="odf", sheet_name="Cupons")

for i in range(len(cupon)):
    cupon['Subtotal'].iloc[i] = Decimal(sub(r'[^\d.]', '', cupon['Subtotal'].iloc[i]))
    cupon['Tax'].iloc[i] = Decimal(sub(r'[^\d.]', '', cupon['Tax'].iloc[i]))
    cupon['Total'].iloc[i] = Decimal(sub(r'[^\d.]', '', cupon['Total'].iloc[i]))
    cupon['Revenue Collected'].iloc[i] = Decimal(sub(r'[^\d.]', '', cupon['Revenue Collected'].iloc[i]))
    cupon['Receivable from Affiliate'].iloc[i] = Decimal(sub(r'[^\d.]', '', cupon['Receivable from Affiliate'].iloc[i]))
    cupon['Payable to Affiliate'].iloc[i] = Decimal(sub(r'[^\d.]', '', cupon['Payable to Affiliate'].iloc[i]))
    cupon['Net Revenue'].iloc[i] = Decimal(sub(r'[^\d.]', '', cupon['Net Revenue'].iloc[i]))
    cupon['% of Report Sum Booking Totals'].iloc[i] = float(sub('%', '', cupon['% of Report Sum Booking Totals'].iloc[i]))

cupon.head()

C:\Users\jpabl\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Item,#,Subtotal,Tax,Total,% of Report Sum Booking Totals,Revenue Collected,Receivable from Affiliate,Payable to Affiliate,Net Revenue
0,Elite Ski Package,1,226.00,13.79,239.79,0.44,239.79,0.00,0.00,234.93
1,Mountain Ski Package,6,1207.55,67.98,1275.53,2.36,1275.53,0.00,0.00,1249.49


In [8]:
#Totals
campaigns['End Subtotal'] = [discount['Subtotal'].sum(), cupon['Subtotal'].sum()]
campaigns['End Tax'] = [discount['Tax'].sum(), cupon['Tax'].sum()]
campaigns['End Total'] = [discount['Total'].sum(), cupon['Total'].sum()]
campaigns['End % of Report Sum Booking Totals'] = [discount['% of Report Sum Booking Totals'].sum(), cupon['% of Report Sum Booking Totals'].sum()]
campaigns['End Revenue Collected'] = [discount['Revenue Collected'].sum(), cupon['Revenue Collected'].sum()]
campaigns['End Receivable from Affiliate'] = [discount['Receivable from Affiliate'].sum(), cupon['Receivable from Affiliate'].sum()]
campaigns['End Payable to Affiliate'] = [discount['Payable to Affiliate'].sum(), cupon['Payable to Affiliate'].sum()]
campaigns['End Net Revenue'] = [discount['Net Revenue'].sum(), cupon['Net Revenue'].sum()]

#Means
campaigns['Mean Subtotal'] = [discount['Subtotal'].mean(), cupon['Subtotal'].mean()]
campaigns['Mean Tax'] = [discount['Tax'].mean(), cupon['Tax'].mean()]
campaigns['Mean Total'] = [discount['Total'].mean(), cupon['Total'].mean()]
campaigns['Mean % of Report Sum Booking Totals'] = [discount['% of Report Sum Booking Totals'].mean(), cupon['% of Report Sum Booking Totals'].mean()]
campaigns['Mean Revenue Collected'] = [discount['Revenue Collected'].mean(), cupon['Revenue Collected'].mean()]
campaigns['Mean Receivable from Affiliate'] = [discount['Receivable from Affiliate'].mean(), cupon['Receivable from Affiliate'].mean()]
campaigns['Mean Payable to Affiliate'] = [discount['Payable to Affiliate'].mean(), cupon['Payable to Affiliate'].mean()]
campaigns['Mean Net Revenue'] = [discount['Net Revenue'].mean(), cupon['Net Revenue'].mean()]


campaigns.head()

,campaign,Start,End,End Subtotal,End Tax,End Total,End % of Report Sum Booking Totals,End Revenue Collected,End Receivable from Affiliate,End Payable to Affiliate,End Net Revenue,Mean Subtotal,Mean Tax,Mean Total,Mean % of Report Sum Booking Totals,Mean Revenue Collected,Mean Receivable from Affiliate,Mean Payable to Affiliate,Mean Net Revenue
0,Save 50% off purchase,12/26/2020 @ 12:00am,1/1/2021 @ 12:00am,49748.63,2798.96,52547.59,97.2,52189.59,0.00,0.00,51134.17,5527.625556,310.995556,5838.621111,10.8,5798.843333,0.0,0.0,5681.574444
1,Skicoupons.com,1/21/21 @ 11:11am,NaN,1433.55,81.77,1515.32,2.8,1515.32,0.00,0.00,1484.42,716.775000,40.885000,757.660000,1.4,757.660000,0.0,0.0,742.210000


# Google analytics

In [9]:
# Read-in aquisition data
aquisition = pd.read_csv("data/HWY40/Google Analytics/aquisition.csv", header=6)

# Transform data types
aquisition['Date'] = [dt.datetime.strptime(str(x), '%Y%m%d') for x in aquisition['Date']]
aquisition['Hour of Day'] = [dt.datetime.strptime(str(x), '%Y%m%d%H') for x in aquisition['Hour of Day']]
aquisition['Organic Searches'] = [int(x) for x in aquisition['Organic Searches']]
aquisition['AdX Clicks'] = [int(x) for x in aquisition['AdX Clicks']]
aquisition['AdX Coverage'] = [float(x.strip('%')) for x in aquisition['AdX Coverage']]
aquisition['AdX CTR'] = [float(x.strip('%')) for x in aquisition['AdX CTR']]
aquisition['AdX eCPM'] = [float(x.strip('$')) for x in aquisition['AdX eCPM']]
aquisition['AdX Impressions'] = [int(x) for x in aquisition['AdX Impressions']]
aquisition['AdX Clicks'] = [int(x) for x in aquisition['AdX Impressions / Session']]
aquisition['AdX Monetized Pageviews'] = [int(x) for x in aquisition['AdX Monetized Pageviews']]
aquisition['AdX Revenue'] = [float(x.strip('$')) for x in aquisition['AdX Revenue']]
aquisition['AdX Revenue / 1000 Sessions'] = [float(x.strip('$')) for x in aquisition['AdX Revenue / 1000 Sessions']]
aquisition['AdX Viewable Impressions %'] = [float(x.strip('%')) for x in aquisition['AdX Viewable Impressions %']]
aquisition.head()

,Date,Hour of Day,Organic Searches,AdX Clicks,AdX Coverage,AdX CTR,AdX eCPM,AdX Impressions,AdX Impressions / Session,AdX Monetized Pageviews,AdX Revenue,AdX Revenue / 1000 Sessions,AdX Viewable Impressions %
0,2021-01-03,2021-01-03 23:00:00,6,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
1,2021-03-17,2021-03-17 20:00:00,6,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
2,2021-01-14,2021-01-14 13:00:00,4,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
3,2021-01-25,2021-01-25 10:00:00,4,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
4,2021-03-10,2021-03-10 18:00:00,4,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0


In [21]:
# Read in behavior data
behavior_1 = pd.read_csv("data/HWY40/Google Analytics/behavior_1.csv", header=6)
behavior_2 = pd.read_csv("data/HWY40/Google Analytics/behavior_2.csv", header=6)
behavior_3 = pd.read_csv("data/HWY40/Google Analytics/behavior_3.csv", header=6)
behaviors = [behavior_1, behavior_2, behavior_3]
behavior = pd.concat(behaviors, ignore_index=True)

# Transform data types
behavior['Date'] = [dt.datetime.strptime(str(x), '%Y%m%d') for x in behavior['Date']]
behavior['Hour of Day'] = [dt.datetime.strptime(str(x), '%Y%m%d%H') for x in behavior['Hour of Day']]
behavior['% Sessions with Search'] = [float(x.strip('%')) for x in behavior['% Sessions with Search']]
behavior['Avg. % Conversion Probability'] = [int(x) for x in behavior['Avg. % Conversion Probability']]
behavior['Avg. Search Probability'] = [int(x) for x in behavior['Avg. Search Depth']]
behavior['Avg. Value'] = [int(x) for x in behavior['Avg. Value']]
behavior['Entrances'] = [int(x) for x in behavior['Entrances']]
behavior['Entrances / Pageviews'] = [float(x.strip('%')) for x in behavior['Entrances / Pageviews']]
behavior['Event Value'] = [int(x) for x in behavior['Event Value']]
behavior['Events / Session with Event'] = [int(x) for x in behavior['Events / Session with Event']]
behavior['Results Pageviews'] = [int(x) for x in behavior['Results Pageviews']]
behavior['Search Depth'] = [int(x) for x in behavior['Search Depth']]
behavior['Search Exits'] = [int(x) for x in behavior['Search Exits']]
behavior['Search Refinements'] = [int(x) for x in behavior['Search Refinements']]
behavior['Sessions with Event'] = [int(x) for x in behavior['Sessions with Event']]
behavior['Sessions with Search'] = [int(x) for x in behavior['Sessions with Search']]
behavior['Time after Search'] = [dt.time(dt.datetime.strptime(str(x), '%H:%M:%S').hour, dt.datetime.strptime(str(x), '%H:%M:%S').minute, dt.datetime.strptime(str(x), '%H:%M:%S').second) for x in behavior['Time after Search']]
behavior['Time on Page'] = [dt.time(dt.datetime.strptime(str(x), '%H:%M:%S').hour, dt.datetime.strptime(str(x), '%H:%M:%S').minute, dt.datetime.strptime(str(x), '%H:%M:%S').second) for x in behavior['Time on Page']]
behavior['Time on Screen'] = [dt.time(dt.datetime.strptime(str(x), '%H:%M:%S').hour, dt.datetime.strptime(str(x), '%H:%M:%S').minute, dt.datetime.strptime(str(x), '%H:%M:%S').second) for x in behavior['Time on Screen']]
behavior['Total Events'] = [int(x) for x in behavior['Total Events']]
behavior['Unique Dimension Combinations'] = [int(x) for x in behavior['Unique Dimension Combinations']]
behavior['Unique Events'] = [int(x) for x in behavior['Unique Events']]
behavior['Avg. Search Probability'] = [int(x) for x in behavior['Avg. Search Probability']]
behavior.tail()

,Date,Hour of Day,Screen Name,% Sessions with Search,Avg. % Conversion Probability,Avg. Search Depth,Avg. Value,Entrances,Entrances / Pageviews,Event Value,Events / Session with Event,Results Pageviews,Search Depth,Search Exits,Search Refinements,Sessions with Event,Sessions with Search,Time after Search,Time on Page,Time on Screen,Total Events,Unique Dimension Combinations,Unique Events,Avg. Search Probability
14077,2021-03-20,2021-03-20 21:00:00,deals.hwy40skidelivery.com/,0.0,0,0,0,1,100.0,0,0,0,0,0,0,0,0,00:00:00,00:00:00,00:00:00,0,1,0,0
14078,2021-03-20,2021-03-20 21:00:00,hwy40skidelivery.com/,0.0,0,0,0,1,100.0,0,0,0,0,0,0,0,0,00:00:00,00:00:37,00:00:00,0,1,0,0
14079,2021-03-20,2021-03-20 21:00:00,hwy40skidelivery.com/packages/,0.0,0,0,0,1,100.0,0,0,0,0,0,0,0,0,00:00:00,00:00:00,00:00:00,0,1,0,0
14080,2021-03-20,2021-03-20 21:00:00,hwy40skidelivery.com/skirentaldelivery,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,00:00:00,00:00:00,00:00:00,0,1,0,0
14081,2021-03-20,2021-03-20 22:00:00,hwy40skidelivery.com/,0.0,0,0,0,1,100.0,0,0,0,0,0,0,0,0,00:00:00,00:00:00,00:00:00,0,1,0,0


In [22]:
# Read in city data
city1 = pd.read_csv("data/HWY40/Google Analytics/city_1.csv", header=6)
city2 = pd.read_csv("data/HWY40/Google Analytics/city_2.csv", header=6)
cities = [city1, city2]
city = pd.concat(cities, ignore_index=True)

# Transform data types
city['Date'] = [dt.datetime.strptime(str(x), '%Y%m%d') for x in city['Date']]
city['Users'] = [int(x) for x in city['Users']]
city['New Users'] = [int(x) for x in city['New Users']]
city['Sessions'] = [int(x) for x in city['Sessions']]
city['Bounce Rate'] = [float(x.strip('%')) for x in city['Bounce Rate']]
city['Pages / Session'] = [float(x) for x in city['Pages / Session']]
city['Avg. Session Duration'] = [dt.time(dt.datetime.strptime(str(x.strip('<')), '%H:%M:%S').hour, dt.datetime.strptime(str(x.strip('<')), '%H:%M:%S').minute, dt.datetime.strptime(str(x.strip('<')), '%H:%M:%S').second) for x in city['Avg. Session Duration']]
city['Transactions'] = [int(x) for x in city['Transactions']]

city.head()

,City,Date,Metro,Region,Users,New Users,Sessions,Bounce Rate,Pages / Session,Avg. Session Duration,Transactions
0,Turner,2021-01-01,Portland-Auburn ME,Maine,33,33,33,0.00,2.94,00:00:08,0
1,Turner,2020-12-31,Portland-Auburn ME,Maine,31,31,31,0.00,3.00,00:00:08,0
2,Rochester,2020-12-31,Rochester NY,New York,19,19,19,5.26,2.84,00:00:08,0
3,Turner,2021-01-02,Portland-Auburn ME,Maine,15,15,15,0.00,2.93,00:00:08,0
4,Clermont,2021-01-01,Orlando-Daytona Beach-Melbourne FL,Florida,14,14,14,0.00,3.00,00:00:09,0


In [23]:
#  Read in device data
device = pd.read_csv("data/HWY40/Google Analytics/device.csv", header=6)

# Transform data types
device['Date'] = [dt.datetime.strptime(str(x), '%Y%m%d') for x in device['Date']]
device['Sessions'] = [int(x) for x in device['Sessions']]
device['Avg. Session Duration'] = [dt.time(dt.datetime.strptime(str(x.strip('<')), '%H:%M:%S').hour, dt.datetime.strptime(str(x.strip('<')), '%H:%M:%S').minute, dt.datetime.strptime(str(x.strip('<')), '%H:%M:%S').second) for x in device['Avg. Session Duration']]
device['Bounce Rate'] = [float(x.strip('%')) for x in device['Bounce Rate']]
device['Users'] = [int(x) for x in device['Users']]
device['Pages / Session'] = [float(x) for x in device['Pages / Session']]
device['Transactions'] = [int(x) for x in device['Transactions']]
device.head()

,Browser,Device Category,Date,Sessions,Avg. Session Duration,Bounce Rate,Users,Pages / Session,Transactions
0,Chrome,desktop,2021-01-25,95,00:00:48,42.11,73,2.57,0
1,Chrome,desktop,2021-02-03,90,00:01:33,50.00,71,2.70,4
2,Chrome,desktop,2021-01-01,80,00:00:09,28.75,75,2.46,0
3,Chrome,desktop,2021-02-02,79,00:01:29,51.90,65,2.48,2
4,Chrome,desktop,2021-02-05,78,00:01:31,53.85,66,2.44,3


In [24]:
# Read in ecommerce data
ecommerce = pd.read_csv("data/HWY40/Google Analytics/ecommerce.csv", header=6)

# Transform data types
ecommerce['Date'] = [dt.datetime.strptime(str(x), '%Y%m%d') for x in ecommerce['Date']]
ecommerce['Hour of Day'] = [dt.datetime.strptime(str(x), '%Y%m%d%H') for x in ecommerce['Hour of Day']]
ecommerce['Avg. QTY'] = [float(x) for x in ecommerce['Avg. QTY']]
ecommerce['Buy-to-Detail Rate'] = [float(x.strip('%')) for x in ecommerce['Buy-to-Detail Rate']]
ecommerce['Cart-to-Detail Rate'] = [float(x.strip('%')) for x in ecommerce['Cart-to-Detail Rate']]
ecommerce['Ecommerce Conversion Rate'] = [float(x.strip('%')) for x in ecommerce['Ecommerce Conversion Rate']]
ecommerce['Product Adds To Cart'] = [int(x) for x in ecommerce['Product Adds To Cart']]
ecommerce['Product Checkouts'] = [int(x) for x in ecommerce['Product Checkouts']]
ecommerce['Product Detail Views'] = [int(x) for x in ecommerce['Product Detail Views']]
ecommerce['Product List CTR'] = [float(x.strip('%')) for x in ecommerce['Product List CTR']]
ecommerce['Product List Clicks'] = [int(x) for x in ecommerce['Product List Clicks']]
ecommerce['Product List Views'] = [int(x) for x in ecommerce['Product List Views']]
ecommerce['Product Refunds'] =  [int(x) for x in ecommerce['Product Refunds']]
ecommerce['Product Removes From Cart'] = [int(x) for x in ecommerce['Product Removes From Cart']]
ecommerce['Quantity'] = [int(x) for x in ecommerce['Quantity']]
ecommerce['Quantity Added To Cart'] = [int(x) for x in ecommerce['Quantity Added To Cart']]
ecommerce['Quantity Checked Out'] = [int(x) for x in ecommerce['Quantity Checked Out']]
ecommerce['Quantity Refunded'] = [int(x) for x in ecommerce['Quantity Refunded']]
ecommerce['Quantity Removed From Cart'] = [int(x) for x in ecommerce['Quantity Removed From Cart']]
ecommerce['Transactions'] = [int(x) for x in ecommerce['Transactions']]
ecommerce['Transactions per User'] = [float(x) for x in ecommerce['Transactions per User']]
ecommerce['Unique Purchases'] = [int(x) for x in ecommerce['Unique Purchases']]

ecommerce.head()

,Date,Hour of Day,Avg. QTY,Buy-to-Detail Rate,Cart-to-Detail Rate,Ecommerce Conversion Rate,Product Adds To Cart,Product Checkouts,Product Detail Views,Product List Clicks,Product List CTR,Product List Views,Product Refunds,Product Removes From Cart,Quantity,Quantity Added To Cart,Quantity Checked Out,Quantity Refunded,Quantity Removed From Cart,Transactions,Transactions per User,Unique Purchases
0,2021-01-18,2021-01-18 20:00:00,4.0,0.0,0.0,50.00,0,0,0,0,0.0,0,0,0,4,0,0,0,0,1,0.33,1
1,2021-03-10,2021-03-10 07:00:00,4.0,0.0,0.0,16.67,0,0,0,0,0.0,0,0,0,4,0,0,0,0,1,0.14,1
2,2021-03-08,2021-03-08 09:00:00,3.0,0.0,0.0,7.69,0,0,0,0,0.0,0,0,0,3,0,0,0,0,1,0.08,1
3,2021-02-26,2021-02-26 17:00:00,5.0,0.0,0.0,100.00,0,0,0,0,0.0,0,0,0,5,0,0,0,0,1,1.00,1
4,2020-12-30,2020-12-30 06:00:00,5.0,0.0,0.0,33.33,0,0,0,0,0.0,0,0,0,5,0,0,0,0,1,0.33,1


In [31]:
# Read in users data
users = pd.read_csv("data/HWY40/Google Analytics/users.csv", header=6)

# Transform data types
users['Date'] = [dt.datetime.strptime(str(x), '%Y%m%d') for x in users['Date']]
users['Hour'] = [int(x) for x in users['Hour']]
users['Avg. Session Duration'] = [dt.time(dt.datetime.strptime(str(x).strip('<'), '%H:%M:%S').hour, dt.datetime.strptime(str(x).strip('<'), '%H:%M:%S').minute, dt.datetime.strptime(str(x).strip('<'), '%H:%M:%S').second) for x in users['Avg. Session Duration']]
users['Avg. Time on Page'] = [dt.time(dt.datetime.strptime(str(x), '%H:%M:%S').hour, dt.datetime.strptime(str(x), '%H:%M:%S').minute, dt.datetime.strptime(str(x), '%H:%M:%S').second) for x in users['Avg. Time on Page']]
users['Avg. Time on Screen'] = [dt.time(dt.datetime.strptime(str(x), '%H:%M:%S').hour, dt.datetime.strptime(str(x), '%H:%M:%S').minute, dt.datetime.strptime(str(x), '%H:%M:%S').second) for x in users['Avg. Time on Screen']]
users['Bounce Rate'] = [float(x.strip('%')) for x in users['Bounce Rate']]
users['Bounces'] = [int(x) for x in users['Bounces']]
users['Exits'] = [int(x) for x in users['Exits']]
users['Hits'] = [int(x) for x in users['Hits']]
users['New Users'] = [int(x) for x in users['New Users']]
users['Number of Sessions per User'] = [float(x) for x in users['Number of Sessions per User']]
users['Pages / Session'] = [float(x) for x in users['Pages / Session']]
users['Pageviews'] = [int(x) for x in users['Pageviews']]
users['Results Pageviews / Search'] = [int(x) for x in users['Results Pageviews / Search']]
users['Screen Views'] = [int(x) for x in users['Screen Views']]
users['Screens / Session'] = [int(x) for x in users['Screens / Session']]
users['Session Duration'] = [dt.time(dt.datetime.strptime(str(x).strip('<'), '%H:%M:%S').hour, dt.datetime.strptime(str(x).strip('<'), '%H:%M:%S').minute, dt.datetime.strptime(str(x).strip('<'), '%H:%M:%S').second) for x in users['Session Duration']]
users['Sessions'] = [int(x) for x in users['Sessions']]
users['Total Unique Searches'] = [int(x) for x in users['Total Unique Searches']]
users['Unique Pageviews'] = [int(x) for x in users['Unique Pageviews']]
users['Unique Screen Views'] = [int(x) for x in users['Unique Screen Views']]
users['Users'] = [int(x) for x in users['Users']]

users.head()

,Date,Hour,Avg. Session Duration,Avg. Time on Page,Avg. Time on Screen,Bounce Rate,Bounces,Exits,Hits,New Users,Number of Sessions per User,Pages / Session,Pageviews,Results Pageviews / Search,Screen Views,Screens / Session,Session Duration,Sessions,Total Unique Searches,Unique Pageviews,Unique Screen Views,Users
0,2020-12-16,11,00:43:11,00:03:18,00:00:00,0.0,0,2,34,1,1.00,8.0,24,0,0,0,02:09:34,3,0,13,0,3
1,2020-12-17,13,00:26:25,00:00:00,00:00:00,0.0,0,0,1,0,1.00,0.0,0,0,0,0,00:26:25,1,0,0,0,1
2,2021-03-11,20,00:24:44,00:12:12,00:00:00,0.0,0,0,2,1,1.00,2.0,2,0,0,0,00:24:44,1,0,2,0,1
3,2020-12-17,15,00:23:47,00:13:37,00:00:00,0.0,0,1,5,0,1.00,1.5,3,0,0,0,00:47:33,2,0,2,0,2
4,2020-12-17,17,00:22:42,00:02:54,00:00:00,0.0,0,3,20,2,1.33,4.0,16,0,0,0,01:30:49,4,0,11,0,3
